<a href="https://colab.research.google.com/github/VickyAlves/Power-BI-Tests/blob/main/Desafio_DIO_Binariza%C3%A7%C3%A3o_de_imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import struct
from PIL import Image  # só para conversão final

def rgb_to_gray(r, g, b):
    """Conversão ponderada RGB → Gray"""
    return int(0.299*r + 0.587*g + 0.114*b)

def carregar_bmp(caminho):
    """Lê uma imagem BMP e retorna cabeçalho e matriz de pixels"""
    with open(caminho, 'rb') as f:
        cabecalho = f.read(54)  # 54 bytes de header
        dados = f.read()

    largura = struct.unpack('<I', cabecalho[18:22])[0]
    altura = struct.unpack('<I', cabecalho[22:26])[0]

    pixels = []
    linha_bytes = (largura * 3 + 3) & ~3
    for y in range(altura):
        linha = []
        for x in range(largura):
            i = y*linha_bytes + x*3
            b, g, r = dados[i], dados[i+1], dados[i+2]
            linha.append((r, g, b))
        pixels.append(linha)

    return cabecalho, pixels, largura, altura

def salvar_bmp(caminho, cabecalho, pixels, largura, altura):
    """Salva imagem BMP a partir da matriz de pixels"""
    with open(caminho, 'wb') as f:
        f.write(cabecalho)
        linha_bytes = (largura * 3 + 3) & ~3
        for y in range(altura):
            linha = []
            for x in range(largura):
                r, g, b = pixels[y][x]
                linha.extend([b, g, r])  # BMP usa BGR
            while len(linha) < linha_bytes:  # padding
                linha.append(0)
            f.write(bytearray(linha))

def converter_para_cinza(pixels, largura, altura):
    """Transforma imagem colorida em grayscale"""
    novo = []
    for y in range(altura):
        linha = []
        for x in range(largura):
            r, g, b = pixels[y][x]
            gray = rgb_to_gray(r, g, b)
            linha.append((gray, gray, gray))
        novo.append(linha)
    return novo

def binarizar(pixels, largura, altura, limiar=128):
    """Transforma grayscale em preto e branco"""
    novo = []
    for y in range(altura):
        linha = []
        for x in range(largura):
            r, g, b = pixels[y][x]
            val = 255 if r > limiar else 0
            linha.append((val, val, val))
        novo.append(linha)
    return novo


# ---- USO ----
# 1) Converta seu PNG original para BMP (pode usar o Pillow só para isso):
entrada_png = "/content/Lenna_(test_image).png"
entrada_bmp = "/content/imagem_temp.bmp"

img = Image.open(entrada_png).convert("RGB")
img.save(entrada_bmp, "BMP")

# 2) Processamento manual
cabecalho, pixels, largura, altura = carregar_bmp(entrada_bmp)

# Grayscale
pixels_gray = converter_para_cinza(pixels, largura, altura)
salvar_bmp("/content/imagem_gray.bmp", cabecalho, pixels_gray, largura, altura)

# Binarização
pixels_bin = binarizar(pixels_gray, largura, altura)
salvar_bmp("/content/imagem_binaria.bmp", cabecalho, pixels_bin, largura, altura)

# 3) Converter BMPs de volta para PNG
Image.open("/content/imagem_gray.bmp").save("/content/imagem_gray.png", "PNG")
Image.open("/content/imagem_binaria.bmp").save("/content/imagem_binaria.png", "PNG")

print("✅ Processamento concluído! Imagens salvas em PNG.")


✅ Processamento concluído! Imagens salvas em PNG.
